*  保存模型权重
    * 方法一：保存checkpoint模型权重

* 保存整个模型
    * 方法二：保存HDF5文件（model.save)
    * 方法三：保存pb文件（tf.saved_model)

`saved_model`格式的模型可以直接用来预测(predict)，但是`saved_model`没有保存优化器配置


方法一仅仅保存了模型中的权重`weights`。

方法二模型和优化器都可以一起保存，包括权重`weights`、模型配置`architecture`和优化器配置`optimizer configuration`。这样做的好处是，当你恢复模型时，完全不依赖于原来搭建模型的代码。

保存完整的模型有很多应用场景，比如在浏览器中使用`TensorFlow.js`加载运行，比如在移动设备上使用`TensorFlow Lite`加载运行。


# Keras版本模型保存与加载

## 保存模型与加载模型

In [1]:
import numpy as np
import tensorflow as tf
x_train = np.random.random((1000, 32))
y_train = np.random.randint(10, size=(1000, ))
x_val = np.random.random((200, 32))
y_val = np.random.randint(10, size=(200, ))
x_test = np.random.random((200, 32))
y_test = np.random.randint(10, size=(200, ))

In [2]:
def get_uncompiled_model():
    inputs = tf.keras.Input(shape=(32,), name='digits')
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
    outputs = tf.keras.layers.Dense(10, name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['sparse_categorical_accuracy'])
    return model


In [3]:
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
32/32 [==============================] - 0s 12ms/step - loss: 2.3145 - sparse_categorical_accuracy: 0.1130 - val_loss: 2.3135 - val_sparse_categorical_accuracy: 0.1250
Epoch 2/5
32/32 [==============================] - 0s 12ms/step - loss: 2.2932 - sparse_categorical_accuracy: 0.1290 - val_loss: 2.3150 - val_sparse_categorical_accuracy: 0.1150
Epoch 3/5
32/32 [==============================] - 0s 11ms/step - loss: 2.2838 - sparse_categorical_accuracy: 0.1370 - val_loss: 2.3205 - val_sparse_categorical_accuracy: 0.1150
Epoch 4/5
32/32 [==============================] - 0s 12ms/step - loss: 2.2743 - sparse_categorical_accuracy: 0.1400 - val_loss: 2.3209 - val_sparse_categorical_accuracy: 0.1100
Epoch 5/5
32/32 [==============================] - 0s 11ms/step - loss: 2.2686 - sparse_categorical_accuracy: 0.1480 - val_loss: 2.3172 - val_sparse_categorical_accuracy: 0.1050


In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 32)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
predictions (Dense)          (None, 10)                650       
Total params: 6,922
Trainable params: 6,922
Non-trainable params: 0
_________________________________________________________________


### 方法一：只保存模型权重

In [6]:
# model.save_weights("adasd.h5")
model.load_weights("adasd.h5")
model.predict(x_test)

array([[-0.09758011, -0.29240972,  0.00580879, ..., -0.01371074,
         0.1409894 ,  0.07672058],
       [-0.10715104, -0.21462788,  0.15357468, ...,  0.31914845,
        -0.05570589,  0.05032081],
       [-0.27643338,  0.00062132,  0.1903002 , ...,  0.33042702,
        -0.19873689, -0.03292914],
       ...,
       [-0.3375801 , -0.01842131,  0.28875068, ...,  0.2708185 ,
         0.03702889, -0.20298825],
       [-0.28116122, -0.18865623,  0.04680794, ...,  0.19834816,
        -0.15128046, -0.03563374],
       [-0.0989755 , -0.1609053 , -0.17766795, ...,  0.39745232,
        -0.02683462, -0.17697851]], dtype=float32)

In [6]:
model.save_weights('./checkpoints/mannul_checkpoint')
model.load_weights('./checkpoints/mannul_checkpoint')
model.predict(x_test)

array([[-4.79599386e-01, -7.43045211e-02,  7.53526539e-02, ...,
        -8.08903947e-02, -1.06541492e-01, -3.22730273e-01],
       [-5.08367836e-01,  6.64272159e-02, -1.10150121e-01, ...,
        -4.87524122e-02, -6.12244979e-02, -1.91938639e-01],
       [-4.95316148e-01, -6.19690008e-02, -1.09398998e-01, ...,
        -8.34675655e-02, -3.04364841e-02, -5.39932489e-01],
       ...,
       [-4.46070462e-01, -1.25298537e-02, -1.24217890e-01, ...,
        -2.40157470e-02,  1.12229452e-01, -5.20460844e-01],
       [-5.37934184e-01, -1.61610134e-02, -1.42415054e-04, ...,
        -8.50361120e-03, -5.21083474e-02, -1.48310274e-01],
       [-3.90737444e-01,  1.99008718e-01,  1.61002487e-01, ...,
         1.07340984e-01,  1.46755893e-02, -4.48317945e-01]], dtype=float32)

### 方法二：保存为pb文件类型

In [8]:
# Export the model to a SavedModel
model.save('keras_model_tf_version', save_format='tf')

# Recreate the exact same model
new_model = tf.keras.models.load_model('keras_model_tf_version')
new_model.predict(x_test)

INFO:tensorflow:Assets written to: keras_model_tf_version\assets


array([[-4.79599386e-01, -7.43045211e-02,  7.53526539e-02, ...,
        -8.08903947e-02, -1.06541492e-01, -3.22730273e-01],
       [-5.08367836e-01,  6.64272159e-02, -1.10150121e-01, ...,
        -4.87524122e-02, -6.12244979e-02, -1.91938639e-01],
       [-4.95316148e-01, -6.19690008e-02, -1.09398998e-01, ...,
        -8.34675655e-02, -3.04364841e-02, -5.39932489e-01],
       ...,
       [-4.46070462e-01, -1.25298537e-02, -1.24217890e-01, ...,
        -2.40157470e-02,  1.12229452e-01, -5.20460844e-01],
       [-5.37934184e-01, -1.61610134e-02, -1.42415054e-04, ...,
        -8.50361120e-03, -5.21083474e-02, -1.48310274e-01],
       [-3.90737444e-01,  1.99008718e-01,  1.61002487e-01, ...,
         1.07340984e-01,  1.46755893e-02, -4.48317945e-01]], dtype=float32)

### 方法三：保存HDF5方式

In [9]:

model.save('keras_model_hdf5_version.h5')

new_model = tf.keras.models.load_model('keras_model_hdf5_version.h5')
new_model.predict(x_test)

array([[-4.79599386e-01, -7.43045211e-02,  7.53526539e-02, ...,
        -8.08903947e-02, -1.06541492e-01, -3.22730273e-01],
       [-5.08367836e-01,  6.64272159e-02, -1.10150121e-01, ...,
        -4.87524122e-02, -6.12244979e-02, -1.91938639e-01],
       [-4.95316148e-01, -6.19690008e-02, -1.09398998e-01, ...,
        -8.34675655e-02, -3.04364841e-02, -5.39932489e-01],
       ...,
       [-4.46070462e-01, -1.25298537e-02, -1.24217890e-01, ...,
        -2.40157470e-02,  1.12229452e-01, -5.20460844e-01],
       [-5.37934184e-01, -1.61610134e-02, -1.42415054e-04, ...,
        -8.50361120e-03, -5.21083474e-02, -1.48310274e-01],
       [-3.90737444e-01,  1.99008718e-01,  1.61002487e-01, ...,
         1.07340984e-01,  1.46755893e-02, -4.48317945e-01]], dtype=float32)

### 方法四：常用于模型部署工作

In [13]:
tf.saved_model.save(model, 'tf_saved_model_version')
restored_saved_model = tf.saved_model.load('tf_saved_model_version')
f = restored_saved_model.signatures["serving_default"]

INFO:tensorflow:Assets written to: tf_saved_model_version\assets


In [14]:
f(digits = tf.constant(x_test.tolist()) )

{'predictions': <tf.Tensor: id=7078, shape=(200, 10), dtype=float32, numpy=
 array([[-4.79599386e-01, -7.43045211e-02,  7.53526539e-02, ...,
         -8.08903947e-02, -1.06541492e-01, -3.22730273e-01],
        [-5.08367836e-01,  6.64272159e-02, -1.10150121e-01, ...,
         -4.87524122e-02, -6.12244979e-02, -1.91938639e-01],
        [-4.95316148e-01, -6.19690008e-02, -1.09398998e-01, ...,
         -8.34675655e-02, -3.04364841e-02, -5.39932489e-01],
        ...,
        [-4.46070462e-01, -1.25298537e-02, -1.24217890e-01, ...,
         -2.40157470e-02,  1.12229452e-01, -5.20460844e-01],
        [-5.37934184e-01, -1.61610134e-02, -1.42415054e-04, ...,
         -8.50361120e-03, -5.21083474e-02, -1.48310274e-01],
        [-3.90737444e-01,  1.99008718e-01,  1.61002487e-01, ...,
          1.07340984e-01,  1.46755893e-02, -4.48317945e-01]], dtype=float32)>}

In [12]:
!saved_model_cli show --dir tf_saved_model_version --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['digits'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32)
        name: serving_default_digits:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['predictions'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


2020-05-19 13:44:40.779279: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


# 自定义版本模型保存与加载

In [15]:
import tensorflow as tf

In [32]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(num_classes)
    
    @tf.function(input_signature=[tf.TensorSpec([None,32], tf.float32, name='digits')])
    def call(self, inputs):
        #定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        return self.dense_2(x)


In [33]:
import numpy as np
x_train = np.random.random((1000, 32))
y_train = np.random.random((1000, 10))
x_val = np.random.random((200, 32))
y_val = np.random.random((200, 10))
x_test = np.random.random((200, 32))
y_test = np.random.random((200, 10))

# 优化器
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# 损失函数
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# 准备metrics函数
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# 准备训练数据集
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# 准备测试数据集
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

In [34]:
model = MyModel(num_classes=10)
epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    # 遍历数据集的batch_size
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # 更新训练集的metrics
        train_acc_metric(y_batch_train, logits)

        # 每200 batches打印一次.
        if step % 200 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
            print('Seen so far: %s samples' % ((step + 1) * 64))

    # 在每个epoch结束时显示metrics。
    train_acc = train_acc_metric.result()
    print('Training acc over epoch: %s' % (float(train_acc),))
    # 在每个epoch结束时重置训练指标
    train_acc_metric.reset_states()

    # 在每个epoch结束时运行一个验证集。
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val)
        # 更新验证集merics
        val_acc_metric(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print('Validation acc: %s' % (float(val_acc),))


Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training loss (for one batch) at step 0: 11.784499168395996
Seen so far: 64 samples
Training acc over epoch: 0.10300000011920929
Validation acc: 0.12999999523162842
Start of epoch 1
Training loss (for one batch) at step 0: 12.349384307861328
Seen so far: 64 samples
Training acc over epoch: 0.10300000011920929
Validation acc: 0.12999999523162842
Start of epoch 2
Training loss (for one batch) at step 0: 12.48736572265625
Seen so far: 64 samples
Training acc over epoch: 0.10300000011920929

### 模型保存方法一

In [19]:
model.save_weights("adasd.h5")
model.load_weights("adasd.h5")
model.predict(x_test)

array([[ 0.59265125,  0.12652864,  0.50779265, ...,  0.38236466,
        -0.48236898, -0.29403484],
       [ 0.3201392 ,  0.31926885,  0.5004305 , ...,  0.34327263,
        -0.46210355, -0.29095772],
       [ 0.17330498,  0.5561173 ,  0.46075135, ...,  0.31273955,
        -0.14429614, -0.6220318 ],
       ...,
       [ 0.45052963,  0.07747109,  0.4848441 , ...,  0.1167865 ,
        -0.53735   , -0.03418449],
       [ 0.2759326 ,  0.5026181 ,  0.52613723, ...,  0.0728192 ,
        -0.3806155 , -0.19111347],
       [ 0.62080455,  1.1130711 ,  0.523637  , ...,  0.31726438,
        -0.10991763, -0.76191264]], dtype=float32)

In [20]:
model.save_weights('./checkpoints/mannul_checkpoint')
model.load_weights('./checkpoints/mannul_checkpoint')
model.predict(x_test)

array([[ 0.59265125,  0.12652864,  0.50779265, ...,  0.38236466,
        -0.48236898, -0.29403484],
       [ 0.3201392 ,  0.31926885,  0.5004305 , ...,  0.34327263,
        -0.46210355, -0.29095772],
       [ 0.17330498,  0.5561173 ,  0.46075135, ...,  0.31273955,
        -0.14429614, -0.6220318 ],
       ...,
       [ 0.45052963,  0.07747109,  0.4848441 , ...,  0.1167865 ,
        -0.53735   , -0.03418449],
       [ 0.2759326 ,  0.5026181 ,  0.52613723, ...,  0.0728192 ,
        -0.3806155 , -0.19111347],
       [ 0.62080455,  1.1130711 ,  0.523637  , ...,  0.31726438,
        -0.10991763, -0.76191264]], dtype=float32)

### 模型保存方法二

In [22]:
# model.save('my_saved_model.h5')


In [ ]:
#new_model = tf.keras.models.load_model('my_saved_model')

In [ ]:
#new_model.predict(x_test)

### 模型保存方法三

In [23]:
model.save('path_to_my_model', save_format='tf')

INFO:tensorflow:Assets written to: path_to_my_model\assets


In [24]:
new_model = tf.keras.models.load_model('path_to_my_model')

In [25]:
new_model.predict(x_test)

array([[ 0.59265125,  0.12652864,  0.50779265, ...,  0.38236466,
        -0.48236898, -0.29403484],
       [ 0.3201392 ,  0.31926885,  0.5004305 , ...,  0.34327263,
        -0.46210355, -0.29095772],
       [ 0.17330498,  0.5561173 ,  0.46075135, ...,  0.31273955,
        -0.14429614, -0.6220318 ],
       ...,
       [ 0.45052963,  0.07747109,  0.4848441 , ...,  0.1167865 ,
        -0.53735   , -0.03418449],
       [ 0.2759326 ,  0.5026181 ,  0.52613723, ...,  0.0728192 ,
        -0.3806155 , -0.19111347],
       [ 0.62080455,  1.1130711 ,  0.523637  , ...,  0.31726438,
        -0.10991763, -0.76191264]], dtype=float32)

### 模型保存方法四

In [35]:
tf.saved_model.save(model, 'my_saved_model')
restored_saved_model = tf.saved_model.load('my_saved_model')  # 没有保存优化器，所以不能直接用predict直接进行预测
f = restored_saved_model.signatures["serving_default"]  # 通过下面第二段的代码，查找对应的字段

INFO:tensorflow:Assets written to: my_saved_model\assets


In [37]:
f(digits = tf.constant(x_test.tolist()) )

{'output_0': <tf.Tensor: id=20472, shape=(200, 10), dtype=float32, numpy=
 array([[0.36048555, 0.77584916, 0.5105363 , ..., 0.31300798, 0.15851551,
         0.9504347 ],
        [0.8945647 , 0.5531711 , 0.34847912, ..., 0.6248529 , 0.36228454,
         1.1269959 ],
        [0.12759946, 0.53442717, 0.7764567 , ..., 0.05553574, 0.31748763,
         0.6228047 ],
        ...,
        [0.59310085, 0.6587581 , 0.19269958, ..., 0.47064906, 0.3493078 ,
         0.650242  ],
        [0.18195015, 0.9256924 , 0.9149718 , ..., 0.34724298, 0.24400793,
         1.0152355 ],
        [0.39257365, 0.8122336 , 1.0617772 , ..., 0.25777856, 0.25043324,
         0.7244146 ]], dtype=float32)>}

In [38]:
!saved_model_cli show --dir my_saved_model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['digits'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32)
        name: serving_default_digits:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


2020-05-19 13:52:56.496039: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll


In [39]:
a = x_test.tolist()[0]

In [47]:
len(a)

32

In [51]:
aa  = list(map(lambda x: int(x),a))

In [ ]:
aa

In [ ]:
f(digits = tf.constant([aa]) )

# 总结

Keras版本保存模型:
* `model.save_weights`（保存模型权重）
* `model.save`（保存模型，本地加载，可保存为h5或者pb格式文件）
* `tf.saved_model.save`（模型部署）


Keras版本加载模型:
* `model.load_weights` （加载模型权重）
* `tf.keras.models.load_model` （加载h5或者pb文件）
* `tf.saved_model.load` （加载模型部署文件）


自定义模型版本保存模型:
* `model.save_weights` (保存模型权重）
* `tf.saved_model.save`（模型部署)
* `model.save`（保存模型，本地加载，可保存为pb格式文件)

自定义模型版本加载模型:
* `model.load_weights` (加载模型权重)
* `tf.saved_model.load`（加载模型部署文件)
* `tf.keras.models.load_model` (加载pb文件)

